## Set up the environment

In [ ]:
import sagemaker as sage
from sagemaker import get_execution_role

role = get_execution_role()
sagemaker_session = sage.Session()


In [ ]:
# Setup variables
model_package_arn='<REPLACE WITH MODEL PACKAGE ARN>'
model_name='customer-lifetime-value'
s3_bucket = '<REPLACE WITH S3 BUCKET NAME>'
sample_file = 'sample.zip'

# Derive S3 paths
s3_input_file_key = 'input/{sample_file}'.format(sample_file=sample_file)
s3_input_dir = 's3://{s3_bucket}/input/'.format(s3_bucket=s3_bucket)
s3_output_dir = 's3://{s3_bucket}/output/'.format(s3_bucket=s3_bucket)
s3_output_path = 's3://{s3_bucket}/output/{sample_file}.out'.format(s3_bucket=s3_bucket, sample_file=sample_file)

print("Model package ARN: ", model_package_arn)
print("Model name: ", model_name)

print("Input file: s3://{s3_bucket}/{s3_input_file_key}".format(s3_bucket=s3_bucket, s3_input_file_key=s3_input_file_key))
print("Input dir: ", s3_input_dir)
print("Output dir: ", s3_output_dir)
print("Output path: ", s3_output_path)

## Realtime Inference

In [ ]:
from sagemaker import ModelPackage

model = ModelPackage(name=model_name,
                     role=role,
                     model_package_arn=model_package_arn,
                     sagemaker_session=sagemaker_session)

In [ ]:
endpoint_name = model_name

In [ ]:
predictor = model.deploy(1, 'ml.m5.large', endpoint_name=endpoint_name)

In [ ]:
# connect to the endpoint
predictor = sage.predictor.RealTimePredictor(
    endpoint_name,
    sagemaker_session=sagemaker_session
)

In [ ]:
# open s3 file
import io
import boto3
import json

s3_client = boto3.client('s3')
s3_response_object = s3_client.get_object(Bucket=s3_bucket, Key=s3_input_file_key)
object_content = s3_response_object['Body'].read()

res = predictor.predict(object_content)

response = json.loads(res)

print(response)

In [ ]:
# Delete endpoint
predictor.delete_endpoint()

## Batch Transform Job

In [ ]:

transformer = sage.transformer.Transformer(
    base_transform_job_name='{model_name}-batch-transform'.format(model_name=model_name),
    model_name=model_name,
    instance_count=1,
    instance_type='ml.m5.large',
    output_path=s3_output_dir,
    sagemaker_session=sagemaker_session
)

In [ ]:
transformer.transform(s3_input_dir)

# wait until prediction job is completed
transformer.wait()


#### Inspect the Batch Transform Output in S3

In [ ]:
s3_client = sage.s3.S3Downloader
s3_client.read_file(s3_output_path)